In [1]:
from numpy import loadtxt
# load array
train = loadtxt('train.csv', delimiter=',')
test = loadtxt('test.csv', delimiter=',')

# Split array
train_x = train[:,:11]
test_x = test[:,:11]
train_y = train[:,11]
test_y = test[:,11]

train_x_noageandsex = train_x[:,:9]
test_x_noageandsex = test_x[:,:9]
train_y_noageandsex = train_y
test_y_noageandsex = test_y

In [2]:
# importing libraries
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Layer, GRU, LSTM, Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from keras.layers import LeakyReLU
from keras import regularizers, backend, initializers
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn import preprocessing
from keras.initializers import Ones, Zeros
import keras.backend as K
import time
import gc
import pandas as pd
import numpy as np
import pylab as plt
import tensorflow as tf
from numpy import savetxt
from tensorflow.python.framework import ops
print(tf.__version__)

#Visualization Libraries
import seaborn as sns

Using TensorFlow backend.


2.1.0


In [3]:
start = time.clock() 
class LayerNormalization(Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)
    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)
    def call(self, x):
        mean = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape

layer_size1 = 10
layer_size2 = 9
layer_size3 = 7
layer_size4 = 5
layer_size5 = 4
layer_size6 = 3
timesteps = 1 # static data
data_dim = 9

X_train_noageandsex = np.reshape(train_x_noageandsex, (train_x_noageandsex.shape[0], 1, train_x_noageandsex.shape[1]))
X_test_noageandsex = np.reshape(test_x_noageandsex, (test_x_noageandsex.shape[0], 1, test_x_noageandsex.shape[1]))
train_y_noageandsex = to_categorical(train_y_noageandsex)

#  use_bias=True, bias_initializer=initializers.Constant(-1)

model = Sequential()
model.add(GRU(layer_size1, return_sequences=True, input_shape=(timesteps, data_dim)))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size2, return_sequences=True))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size3, return_sequences=True))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size4, return_sequences=True))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size5, return_sequences=True))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(GRU(layer_size6, return_sequences=False))
model.add(LayerNormalization())
model.add(LeakyReLU(alpha=0.01))
model.add(Dense(2, activation='softmax'))


opt = optimizers.Adam(0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# Stop the training when there is no improvement in the validation accuracy for ten consecutive epochs.
early = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)

# reduces learning rate when a metric has stopped improving
redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=7, verbose=0)

# defining the callbacks list to include the above parameters
callbacks_list = [early, redonplat]

# train the model
history = model.fit(X_train_noageandsex, train_y_noageandsex, epochs=1000, verbose=0, callbacks=callbacks_list, validation_split=0.1)
end = time.clock()
print("Time for training: {} ".format((end-start))) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Epoch 00068: early stopping
Time for training: 6593.4894758 


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [4]:
# test the model
start = time.clock() 
pred_test = model.predict(X_test_noageandsex)
end = time.clock()
pred_test = np.argmax(pred_test, axis=-1)
print("Time for prediction: {} ".format((end-start))) 


# get f1 score of the model & print it. The f1 score considers the precision & recall.
f1 = f1_score(test_y_noageandsex, pred_test, average="macro")
print("Test f1 score : %s "% f1)

# get ROC AUC score of the model & print it
roc = roc_auc_score(test_y_noageandsex, pred_test)
print("Test ROC AUC Score : %s "% roc)

# get the accuracy and print it
acc = accuracy_score(test_y_noageandsex, pred_test)
print("Test accuracy score : %s "% acc)

# Save the model
model.save("model_noageandsex1_final.h5")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


Time for prediction: 5.749084199999743 
Test f1 score : 0.770089366791743 
Test ROC AUC Score : 0.7633939364867299 
Test accuracy score : 0.8013578169027402 


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.
